In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -qq /content/gdrive/MyDrive/cats_vs_dogs_small_dataset.zip

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from keras import layers
from keras import models
import matplotlib.pyplot as plt

In [ ]:
# Code to remove the data and folder created from a previous run of the data
!rm -rf "cats_vs_dogs_small"
#!rm -rf "cats_vs_dogs_small_dataset"

In [ ]:
# Use the following code to partition the data into train/test/validation set:
import os, shutil, pathlib

original_dir = pathlib.Path("cats_vs_dogs_small_dataset")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    count = 0
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{i}.jpg" for i in range(start_index, end_index)]
        for fname in os.listdir(original_dir / category)[start_index:end_index]:
            shutil.copyfile(original_dir / category / fname, dir / fname)
            count += 1
    return count

train_count = make_subset("train", start_index=0, end_index=1500)
validation_count = make_subset("validation", start_index=1500, end_index=1750)
test_count = make_subset("test", start_index=1750, end_index=2000)

print(f"Number of images in train subset: {train_count}")
print(f"Number of images in validation subset: {validation_count}")
print(f"Number of images in test subset: {test_count}")

train_ds = keras.utils.image_dataset_from_directory(
    directory = pathlib.Path("cats_vs_dogs_small/train"),
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(150,150)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = pathlib.Path("cats_vs_dogs_small/test"),
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(150,150)


In [ ]:
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
#model.add(layers.Dropout(0.3))  # Added dropout here
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()